In [31]:
import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import overturemaps
from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_continuous_cmap
from matplotlib import colormaps
from matplotlib.colors import LogNorm
from shapely import box


# Buildings data

Read [Overture Maps building data](https://docs.overturemaps.org/guides/buildings/#14/32.58453/-117.05154/0/60) within a 10km buffer of London.

In [38]:
bbox = (-0.354309,51.394922,0.122910,51.606077)

poly = box(*bbox)

x = poly.centroid.x
y = poly.centroid.y

Read buildings data from `bbox`.

In [39]:
table = overturemaps.record_batch_reader("building", bbox).read_all()

table = table.combine_chunks()

In [40]:
data = table.to_pandas()

gdf = gpd.GeoDataFrame(data, geometry=gpd.GeoSeries.from_wkb(data.geometry, crs=4326))

Here we're doing a roundabout way of getting it into a `geopandas.GeoDataFrame`. You can use:

```python
    gdf = overturemaps.core.geodataframe("building", bbox=bbox)
```

But I've noticed this timeouts out quicker than the above method.

In [43]:
# 880K polygons
len(gdf)

879670

In [44]:
heights = gdf["height"].to_numpy()
heights = np.nan_to_num(heights, nan=1)

normalizer = LogNorm(1, heights.max(), clip=True)
normalized_heights = normalizer(heights)

colors = apply_continuous_cmap(normalized_heights, colormaps["plasma"])

In [45]:
building_layer = PolygonLayer.from_geopandas(
    gdf=gdf[["id", "height", "geometry", "names"]],
    extruded=True,
    get_elevation=heights,
    get_fill_color=colors,
)

In [50]:
view_state = {
    "longitude": x,
    "latitude": y,
    "zoom": 11,
    "pitch": 90,
    "bearing": 0,
}

map_layout = widgets.Layout(height="1200px")

m = Map(building_layer, view_state=view_state, layout=map_layout)

m

Map(layers=[PolygonLayer(extruded=True, get_elevation=<pyarrow.lib.FloatArray object at 0x33b7693c0>
[
  3.284…

In [47]:
m.to_html("examples/buildings.html")